# Working with Data using Spark and Python
## Creating and Working with a RDDs (Resillient Distributed Datasets)
In this section of the Notebook, we will first use Spark to interactively explore __Wikipedia__ data.  
__Note:__ For the purposes of reproducibility, the data can be downloaded from [here](http://d2xijq8wc2iink.cloudfront.net/ampcamp6/ampcamp6-rc3.tar.gz).

To start, we create an RDD named `pagecounts` from the input files in the `data` directory. Since the `SparkContext` is already created as variable `sc`, the first task is view this to confirm before creating the RDD.  
__Note:__ The `pagecounts` data has already been copied to __HDFS__ and thus shared between the cluster nodes.  

In [1]:
sc

In [2]:
# Create pagecounts variable
pagecounts = sc.textFile("hdfs://master:54310/data/pagecounts")

To view the data, we use the `take` operation of an RDD to get the first __K__ record were, K = 10.

In [3]:
pagecounts.take(10)

[u'20090505-000000 aa Main_Page 2 9980',
 u'20090505-000000 ab %D0%90%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82 1 465',
 u'20090505-000000 ab %D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0%B4%D0%B0%D2%9F%D1%8C%D0%B0 1 16086',
 u'20090505-000000 af.b Tuisblad 1 36236',
 u'20090505-000000 af.d Tuisblad 4 189738',
 u'20090505-000000 af.q Tuisblad 2 56143',
 u'20090505-000000 af Afrika 1 46833',
 u'20090505-000000 af Afrikaans 2 53577',
 u'20090505-000000 af Australi%C3%AB 1 132432',
 u'20090505-000000 af Barack_Obama 1 23368']

As we can see from the output above, the data is not easy to understand. This is because `take()` returns an array and Python simply prints the array with each element separated by a `,`. To alleviate this we can traverse the and print each record on its own line.

In [4]:
for i in pagecounts.take(10):
    print i

20090505-000000 aa Main_Page 2 9980
20090505-000000 ab %D0%90%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82 1 465
20090505-000000 ab %D0%98%D1%85%D0%B0%D0%B4%D0%BE%D1%83_%D0%B0%D0%B4%D0%B0%D2%9F%D1%8C%D0%B0 1 16086
20090505-000000 af.b Tuisblad 1 36236
20090505-000000 af.d Tuisblad 4 189738
20090505-000000 af.q Tuisblad 2 56143
20090505-000000 af Afrika 1 46833
20090505-000000 af Afrikaans 2 53577
20090505-000000 af Australi%C3%AB 1 132432
20090505-000000 af Barack_Obama 1 23368


Although not much better, the above output elminites some of the array specifics to give us better view of the actual data within context. To further explore the data, we can subset or filter the based on specific criteria. In the following example, we will filter only the English pages (en). This is done by applying the `filter()` function to `pagecounts`. So for each record, we can split it by the field delimiter (i.e. a space) and get the second field (project code) and compare it with the string “en”. 

__TIP:__ To avoid reading from disks each time we perform any operations on the RDD, we also cache the RDD into memory. This is where Spark really starts to to shine. 

In [5]:
enPages = pagecounts.filter(lambda x: x.split(" ")[1] == "en").cache()

Using the `filter()` function is a lazy evaluation, so no computation is done yet. Next time any action is invoked on `enPages`, Spark will cache the data set in memory across the workers in your cluster. For example, we can view the total records of pages using the `count()` function and then compare that to the total number of records for English pages.

In [6]:
print "Total number of pages:",pagecounts.count()
print "Total numbner of English pages:", enPages.count()

Total number of pages: 1398882
Total numbner of English pages: 970545


Next we can generate a histogram of total page views on __Wikipedia__ English pages for a specific data range represented in our dataset (i.e. May 5 to May 7, 2009). The high level idea is as follows: 
1. Generate a key value pair for each line, the key is the date (the first eight characters of the first field), and the value is the number of pageviews for that date (the fourth field). 
2. Shuffle the data and group all values of the same key together. 
3. Sum up the values for each key. There is a convenient method called reduceByKey in Spark for exactly this pattern. By default, Spark assumes that the reduce function is commutative and associative and applies combiners on the mapper side. 

The `collect` method at the end converts the result from an RDD to an array.

In [7]:
# Split the Data
enTuples = enPages.map(lambda x: x.split(" "))

# Generate <Key><Value> pairs
enKeyValuePairs = enTuples.map(lambda x: (x[0][:8], int(x[3])))

# Shuffle the data and group all values of the same key together
enKeyValuePairs.reduceByKey(lambda x, y: x + y, 1).collect()

[(u'20090507', 6175726), (u'20090505', 7076855)]

__Tip:__ We can combine the previous three commands into one:

In [8]:
enPages.map(lambda x: x.split(" ")).map(lambda x: (x[0][:8], int(x[3]))).reduceByKey(lambda x, y: x + y, 1).collect()

[(u'20090507', 6175726), (u'20090505', 7076855)]

Suppose we want to find pages that were viewed more than 200,000 times during the three days covered by our dataset. Conceptually, this task is similar to the previous query. But, given the large number of pages (23 million distinct page names), the new task is very expensive. We are doing an expensive group-by with a lot of network shuffling of data. To recap: 
1. We split each line of data into its respective fields. 
2. We extract the fields for page name and number of page views. We reduce by key again, this time with 40 reducers. 
3. We filter out pages with less than 200,000 total views over our time window represented by our dataset.

In [9]:
enPages.map(lambda x: x.split(" ")) \
.map(lambda x: (x[2], int(x[3]))) \
.reduceByKey(lambda x, y: x + y, 40) \
.filter(lambda x: x[1] > 200000) \
.map(lambda x: (x[1], x[0])) \
.collect()

[(451126, u'Main_Page'), (1066734, u'404_error/'), (468159, u'Special:Search')]

## Data Exploration using Spark SQL
Spark SQL is the newest component of Spark and provides a SQL like interface. Spark SQL is tightly integrated with the the various spark programming languages.

In [10]:
# Verify the sqlContext
sqlContext

Note how the `sqlContext` is like the `HiveContect`, meaning that the use of SQL in this context is very similar to using SQL against a Hive Metastore in Hadoop. As in that case, where SQL commands are translated to MapReduce jobs, our SQL commands are translated to working RDD's.

To start we load a data frame in that is stored in the __Parquet__ format. Parquet is a self-describing columnar file format. Since it is self-describing, Spark SQL will automatically be able to infer all of the column names and their datatypes. For this part of the Notebook, we will use a set of data that contains all of the pages on wikipedia that contain the word “berkeley”. We will load this data using the input methods provided by SQLContext.  
__Note:__ The data has already been loaded onto __HDFS__.

In [11]:
# Load the data from HDFS
wikiData = sqlContext.read.parquet("hdfs://master:54310/data/wiki.parquet")

The result of the above command is a SchemaRDD. A SchemaRDD has all of the functions of a normal RDD, for example, lets figure out how many records are in the data set.

In [12]:
wikiData.count()

39365

In addition to standard RDD operatrions, SchemaRDDs also have extra information about the names and types of the columns in the dataset, just like the schema of RDBMS table. So basically, SchemaRDD = RDD [Row] + Schema. This extra schema information makes it possible to run SQL queries against the data after it has been registered as a table. Below is an example of counting the number of records using a SQL query.

In [13]:
# Create a temporary table for SQL queries
wikiData.registerTempTable("wikiData")

# SQL Query to count
result = sqlContext.sql("SELECT COUNT(*) AS pageCount FROM wikiData").collect()
print result

[Row(pageCount=39365)]


The result of SQL queries is always a collection of Row objects. From a row object we can access the individual columns of the result.

In [14]:
result[0].pageCount

39365

SQL can be a powerful tool from performing complex aggregations. For example, the following query returns the top 10 usersnames by the number of pages they created.

In [15]:
sqlContext. \
sql("SELECT username, COUNT(*) AS cnt FROM wikiData WHERE username <> '' GROUP BY username ORDER BY cnt DESC LIMIT 10") \
.collect()

[Row(username=u'Waacstats', cnt=2003),
 Row(username=u'Cydebot', cnt=949),
 Row(username=u'BattyBot', cnt=939),
 Row(username=u'Yobot', cnt=890),
 Row(username=u'Addbot', cnt=853),
 Row(username=u'Monkbot', cnt=668),
 Row(username=u'ChrisGualtieri', cnt=438),
 Row(username=u'RjwilmsiBot', cnt=387),
 Row(username=u'OccultZone', cnt=377),
 Row(username=u'ClueBot NG', cnt=353)]

## Data Exploration using Spark DataFrames
Now that we have a very basic idea of working with RDD's and using SQL queries against them, wwe next look at __Spark DataFrames__. The DataFrames API is inspired by data frames in `R` and `Python` (Pandas) and is used for storing tabular data, along with labeled axes for both rows and columns. Since the concept was introduced in `R` before it was in `Python` Pandas the methadology of working with DataFrames can be easly applied to Spark. DataFrames can be constructed from a wide array of sources such as: 
- structured data files
- tables in Hive
- external databases
- existing RDDs.

It is important however, to be cogniscient of some of the differences between data frames in `R` and `Python`. In `R`, a `data.frame()` is a list of vector variables of the same number of elements (rows) with unique row names. So, each column is a vector with an associated name, and each row is a series of vector elements that correspond to the same position in each of the column-vectors.

In `Python` Pandas, a DataFrame can be thought of as a dict-like container for Series objects, where a Series is a one-dimensional NumPy `ndarray` with axis labels (including time series). By default, each Series correspond with a column in the resulting DataFrame. To elaborate, we will show some exammples of using them with some basic statistical and mathamatical functions.

### Random Number Generation  
Random data generation is useful for testing of existing algorithms and implementing randomized algorithms, such as random projection. The following methods are under `sql.functions` for generating columns that contains Independent and identically distributed random variables (i.i.d.) values drawn from a distribution, e.g., uniform `rand`,  and standard normal `randn`.

In [16]:
from pyspark.sql.functions import rand, randn
# Create a DataFrame with one int column and 10 rows.
df = sqlContext.range(0, 10)
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [17]:
# Generate two other columns using uniform distribution and normal distribution.
df.select("id", rand(seed=10).alias("uniform"), randn(seed=27).alias("normal")).show()

+---+-------------------+--------------------+
| id|            uniform|              normal|
+---+-------------------+--------------------+
|  0|0.27149331793166864|-0.18575112254167045|
|  1| 0.8469085276976173|  0.4932212707592389|
|  2| 0.8122802274304282|  1.2423430583597714|
|  3|  0.869635429461509|-0.16445018569360123|
|  4|0.46517581696992505|  -0.705186286277025|
|  5| 0.8642043127063618|  0.3900018344856156|
|  6|0.37523872200244035|  0.5687916150439113|
|  7|0.24420989078856126|  -1.979872827161101|
|  8| 0.8292577771850476|  1.8077401259195247|
|  9| 0.9690555459609131|-0.22986601831364423|
+---+-------------------+--------------------+



### Descriptive and Summary Statistics  
Before performing any type of exploratory data analysis, typicall the first thing to do is to get an idea of what the data looks like. Usually, for numerical columns, knowing the descriptive summary statistics can help a lot in understanding the distribution of the data. In `R`, this is typically done using the `summary()` and `str()` functions. The following `python` code returns a DataFrame containing information such as number of non-null entries (count), mean, standard deviation, and minimum and maximum value for each numerical column. 

In [18]:
# A slightly different way to generate the two random columns
df = sqlContext.range(0, 10).withColumn('uniform', rand(seed=10)).withColumn('normal', randn(seed=27))
df.describe().show()

+-------+------------------+-------------------+------------------+
|summary|                id|            uniform|            normal|
+-------+------------------+-------------------+------------------+
|  count|                10|                 10|                10|
|   mean|               4.5| 0.6547459568134472| 0.123697146458102|
| stddev|3.0276503540974917| 0.2809749416672925|1.0492745213734231|
|    min|                 0|0.24420989078856126|-1.979872827161101|
|    max|                 9| 0.9690555459609131|1.8077401259195247|
+-------+------------------+-------------------+------------------+



Should the DataFrame be large, we can also run describe on a subset of the columns:

In [19]:
df.describe('uniform', 'normal').show()

+-------+-------------------+------------------+
|summary|            uniform|            normal|
+-------+-------------------+------------------+
|  count|                 10|                10|
|   mean|  0.654745956813447| 0.123697146458102|
| stddev| 0.2809749416672925|1.0492745213734231|
|    min|0.24420989078856126|-1.979872827161101|
|    max| 0.9690555459609131|1.8077401259195247|
+-------+-------------------+------------------+



While the `describe()` function works well for exploratory data analysis, we can also generate a list of descriptive statistics and the columns they apply to using the normal select on a DataFrame:

In [20]:
from pyspark.sql.functions import mean, min, max
df.select([mean('uniform'), min('uniform'), max('uniform')]).show()

+-----------------+-------------------+------------------+
|     avg(uniform)|       min(uniform)|      max(uniform)|
+-----------------+-------------------+------------------+
|0.654745956813447|0.24420989078856126|0.9690555459609131|
+-----------------+-------------------+------------------+



### Covariance and Correlation  
[Covariance](https://en.wikipedia.org/wiki/Covariance) is a measure of how two variables change with respect to each other. A *positive* number would mean that there is a tendency that as one variable __increases__, the other __increases__ as well. A *negative* number would mean that as one variable __increases__, the other variable has a tendency to __decrease__. The sample covariance of two columns of a DataFrame can be calculated as follows:

In [21]:
df = sqlContext.range(0, 10).withColumn('rand1', rand(seed=10)).withColumn('rand2', rand(seed=27))
df.show()
print "Covariance of the two columns is:", df.stat.cov('rand1', 'rand2')
print "Covariance of the `id` column with itself is:", df.stat.cov('id', 'id')

+---+-------------------+--------------------+
| id|              rand1|               rand2|
+---+-------------------+--------------------+
|  0|0.27149331793166864|  0.3071632607232556|
|  1| 0.8469085276976173|  0.9662041723146931|
|  2| 0.8122802274304282|  0.8244413413402464|
|  3|  0.869635429461509|  0.4030672316912547|
|  4|0.46517581696992505|  0.3944153996903591|
|  5| 0.8642043127063618|  0.5371937843819895|
|  6|0.37523872200244035|  0.8874450461989395|
|  7|0.24420989078856126|  0.3141703608698413|
|  8| 0.8292577771850476|0.014167297219329189|
|  9| 0.9690555459609131|   0.746171676032295|
+---+-------------------+--------------------+

Covariance of the two columns is: 0.0202406512194
Covariance of the `id` column with itself is: 9.16666666667


As can be seen from the above, the covariance of the two randomly generated columns, __rand1__ and __rand2__, is close to zero, while the covariance of the __id__ column with itself is very high. But what does the value of __9.17__ mean? That's where [Correlation](https://en.wikipedia.org/wiki/Correlation_and_dependence) comes into play. Correlation is a normalized measure of covariance that is easier to understand, as it provides a quantitative measurement of the statistical dependence between two random variables.

In [22]:
print "Correlation of the two columns is:", df.stat.corr('rand1', 'rand2')
print "Correlation of the `id` colum with itslef is:", df.stat.corr('id', 'id')

Correlation of the two columns is: 0.234759219635
Correlation of the `id` colum with itslef is: 1.0


So, as can be seen, the __id__ column correlates perfectly with itself, while the two randomly generated columns have low correlation value.

### Cross Tabulation  
Cross Tabulation or [Contingency Tables](https://en.wikipedia.org/wiki/Contingency_table) is a table showing the freqency distribution for a set of variables and is therefore a powerful tool statistical that is used to observe the statistical significance (or independence) of variables. Usinf Spark DataFrames, we can obtain the frequency counts of the different pairs that are observed with the columns. 

__Note:__ The cardinality of columns we run crosstab on cannot be too big. That is to say, the number of distinct “name” and “item” cannot be too large. For example, if “item” contains 1 billion distinct entries, there would be no way to do exploratory data analysis as the resultant table would never fit onthe screen.

In [23]:
# Create a DataFrame with two columns (name, item)
names = ["Alice", "Bob", "Mike"]
items = ["milk", "bread", "butter", "apples", "oranges"]
df = sqlContext.createDataFrame([(names[i % 3], items[i % 5]) for i in range(100)], ["name", "item"])

# Display the first 10 records
df.show(10)

+-----+-------+
| name|   item|
+-----+-------+
|Alice|   milk|
|  Bob|  bread|
| Mike| butter|
|Alice| apples|
|  Bob|oranges|
| Mike|   milk|
|Alice|  bread|
|  Bob| butter|
| Mike| apples|
|Alice|oranges|
+-----+-------+
only showing top 10 rows



In [24]:
# Generate the Contingency Table
df.stat.crosstab("name", "item").show()

+---------+------+-------+------+----+-----+
|name_item|apples|oranges|butter|milk|bread|
+---------+------+-------+------+----+-----+
|      Bob|     6|      7|     7|   6|    7|
|     Mike|     7|      6|     7|   7|    6|
|    Alice|     7|      7|     6|   7|    7|
+---------+------+-------+------+----+-----+



### Element Frequency  
Knowing the frequency of the occurance of data can be useful in understing the data. Using DataFrames, we can find the frequent items for a set of columns. Spark uses a [one-pass algorithm](http://dl.acm.org/citation.cfm?doid=762471.762473) proposed by Karp et al. This is a fast, approximate algorithm that always return all the frequent items that appear in a user-specified minimum proportion of rows. 

__Note:__ The result might contain false positives, i.e. items that are not frequent. 

In [25]:
df = sqlContext.createDataFrame([(1, 2, 3) if i % 2 == 0 else (i, 2 * i, i % 4) for i in range(100)], ["a", "b", "c"])
df.show(10)

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  1|  2|  1|
|  1|  2|  3|
|  3|  6|  3|
|  1|  2|  3|
|  5| 10|  1|
|  1|  2|  3|
|  7| 14|  3|
|  1|  2|  3|
|  9| 18|  1|
+---+---+---+
only showing top 10 rows



In [26]:
# Given the above DataFrame, the following code finds the frequent items that show up 40% of the time for each column
freq = df.stat.freqItems(["a", "b", "c"], 0.4)
freq.collect()[0]

Row(a_freqItems=[65, 1], b_freqItems=[2, 130], c_freqItems=[1, 3])

As can be seen from the outpu above, __11__ and __1__ are the frequent values for column __a__. We can also find frequent items for column combinations, by creating a composite column using the `struct()` function:

In [27]:
from pyspark.sql.functions import struct
freq = df.withColumn('ab', struct('a', 'b')).stat.freqItems(['ab'], 0.4)
freq.collect()[0]

Row(ab_freqItems=[Row(a=9, b=18), Row(a=1, b=2)])

From the above output, the combination of __a=11__ and __b=22___, and __a=1__ and __b=2__ appear frequently in this dataset.  

__Note__ that “a=11 and b=22” is a false positive.

### Mathematical Functions  
A number of methmatical functions can also be applied to rows and columns in a DataFrame. The inputs need to be columns functions that take a single argument, such as __cos__, __sin__, __floor__ and __ceil__. For functions that take two arguments as input, such as __pow__, __hypot__, either two columns or a combination of a double and column can be supplied, for example:

In [28]:
from pyspark.sql.functions import *
df = sqlContext.range(0, 10).withColumn('uniform', rand(seed=10) * 3.14)

# Reference a column or supply the column name
df.select('uniform',
          toDegrees('uniform'),
          (pow(cos(df['uniform']), 2) + pow(sin(df.uniform), 2)). \
          alias("cos^2 + sin^2")).show()

+------------------+------------------+------------------+
|           uniform|  DEGREES(uniform)|     cos^2 + sin^2|
+------------------+------------------+------------------+
|0.8524890183054395| 48.84402283015247|0.9999999999999999|
|2.6592927769705184|152.36625261003522|               1.0|
|2.5505599141315445|146.13631847498715|               1.0|
| 2.730655248509138| 156.4550210448206|               1.0|
|1.4606520652855648| 83.68919867793005|               1.0|
| 2.713601541897976|155.47791563094665|               1.0|
|1.1782495870876628| 67.50872855315502|0.9999999999999999|
|0.7668190570760823|  43.9354956206609|               1.0|
|2.6038694203610495|149.19072818986416|0.9999999999999999|
| 3.042834414317267| 174.3415696975411|1.0000000000000002|
+------------------+------------------+------------------+



## Conclusion  
Now that we have a small overview of how to do some basic data exploration using Spark, we can apply these principles and do some exploratory data analysis.